In [10]:
import pyvista as pv
import os

custom_palette = {
    'roze': '#eb8fd8',
    'groen': '#b9d4b4',
    'paars': '#ba94e9',
    'blue': '#4C8BE2',
    'orange': '#E27A3F',
    'grey_light': '#1F3240',
    'grey_dark': '#16242F'
}

In [11]:
def create_structured_grid(volume_np):
    # Get the dimensions of the volume
    W, H, D = volume_np.shape
    
    # Create grid coordinates
    x = np.arange(W)
    y = np.arange(H)
    z = np.arange(D)
    
    # Generate the meshgrid
    X, Y, Z = np.meshgrid(x, y, z, indexing='ij')
    
    # Create a structured grid
    grid = pv.StructuredGrid(X, Y, Z)
    
    # Assign the label data to 'point_data'
    grid.point_data["labels"] = volume_np.flatten(order="F")
    
    return grid

# Functions

In [12]:
def render_3d_segmentation(pred_volume, gt_volume, volume_id, output_dir, view_type):
    pred_volume_np = torch.argmax(pred_volume, dim=0).cpu().numpy()
    gt_volume_np = torch.argmax(gt_volume, dim=0).cpu().numpy()
    pred_volume_np = np.transpose(pred_volume_np, (2, 1, 0))
    gt_volume_np = np.transpose(gt_volume_np, (2, 1, 0))

    grid_pred = create_structured_grid(pred_volume_np)
    grid_gt = create_structured_grid(gt_volume_np)

    class_names = ['Background', 'Esophagus', 'Heart', 'Trachea', 'Aorta']
    class_colors = ['gray', custom_palette['roze'], custom_palette['groen'], custom_palette['paars'], custom_palette['blue']]

    p = pv.Plotter(shape=(1, 2), window_size=(1600, 800), off_screen=True)

    def add_class_surfaces(grid, subplot_index, title, view_type):
        p.subplot(0, subplot_index)
        for c in range(1, len(class_names)):
            class_label = c
            class_grid = grid.threshold(value=(class_label - 0.1, class_label + 0.1), scalars='labels')
            if class_grid.n_points == 0:
                continue
            surface = class_grid.contour(isosurfaces=[class_label], scalars='labels')
            p.add_mesh(surface, color=class_colors[c], opacity=0.6, label=class_names[c])
        p.add_legend(bcolor='white', loc='lower right' )
        p.add_axes()
        p.set_background('white')
        # Update title with view_type
        p.add_title(f'{title} - {view_type}')

    # Add ground truth and prediction surfaces with view_type in the title
    add_class_surfaces(grid_gt, subplot_index=0, title='Ground Truth', view_type=view_type)
    add_class_surfaces(grid_pred, subplot_index=1, title='Prediction', view_type=view_type)
    
    p.link_views()

    camera_positions = {'isometric': 'iso', 'front': 'xz', 'side': 'yz', 'top': 'xy'}
    for view_name, camera_pos in camera_positions.items():
        p.camera_position = camera_pos
        p.render()
        screenshot_path = os.path.join(output_dir, f'{volume_id}_{view_type}_{view_name}_view.png')
        p.screenshot(screenshot_path)
        print(f"Saved {view_name} view to {screenshot_path}")

    p.close()
    print(f"Rendered 3D segmentation saved to {screenshot_path}")

In [13]:
import nibabel as nib
import numpy as np
import torch
from scipy.ndimage import zoom

def load_nifti_as_tensor(nifti_path, num_classes):
    """
    Loads a NIfTI file and converts it into a one-hot encoded PyTorch tensor.
    Resamples the data to have isotropic voxel sizes to correct aspect ratio distortion.

    Args:
        nifti_path (str): Path to the NIfTI file (.nii or .nii.gz).
        num_classes (int): Total number of classes (including background).

    Returns:
        tensor_volume (torch.Tensor): One-hot encoded tensor of shape (K, D, H, W).
    """
    # Load the NIfTI file
    nifti_img = nib.load(nifti_path)
    volume_data = nifti_img.get_fdata()

    # Print some information
    print(f"Loaded NIfTI file from {nifti_path}")
    print(f"Original volume shape: {volume_data.shape}")
    print(f"Data type: {volume_data.dtype}")

    # Reorient the image to RAS+ (standard orientation)
    canonical_img = nib.as_closest_canonical(nifti_img)
    volume_data = canonical_img.get_fdata()

    # Get voxel sizes from the canonical image
    voxel_sizes = canonical_img.header.get_zooms()
    print(f"Voxel sizes (after reorientation): {voxel_sizes}")

    # Get axes codes after reorientation
    aff = canonical_img.affine
    axes_codes = nib.orientations.aff2axcodes(aff)
    print(f"Axes codes after reorientation: {axes_codes}")

    # Transpose the data to match the expected (D, H, W) shape
    volume_data = np.transpose(volume_data, (2, 1, 0))  # Now shape is (D, H, W)
    print(f"Volume shape after transpose: {volume_data.shape}")

    # Resample the data to isotropic voxel sizes
    target_voxel_size = min(voxel_sizes)
    scaling_factors = (
        voxel_sizes[2] / target_voxel_size,  # scaling factor for D (Z-axis)
        voxel_sizes[1] / target_voxel_size,  # scaling factor for H (Y-axis)
        voxel_sizes[0] / target_voxel_size   # scaling factor for W (X-axis)
    )
    print(f"Scaling factors: {scaling_factors}")

    # Resample the data using nearest-neighbor interpolation
    volume_data_resampled = zoom(volume_data, zoom=scaling_factors, order=0)
    print(f"Volume shape after resampling: {volume_data_resampled.shape}")

    # Convert volume data to integer type
    volume_data_resampled = volume_data_resampled.astype(np.int32)

    # Determine the unique labels in the resampled volume
    unique_labels = np.unique(volume_data_resampled)
    print(f"Unique labels in the resampled volume: {unique_labels}")

    # Create one-hot encoded tensor
    K = num_classes
    D, H, W = volume_data_resampled.shape
    tensor_volume = torch.zeros((K, D, H, W), dtype=torch.float32)

    # Fill the tensor with one-hot encoding
    for k in range(num_classes):
        tensor_volume[k] = torch.from_numpy((volume_data_resampled == k).astype(np.float32))

    return tensor_volume


# old code

In [14]:
# # gt_nifti_path = '../../data/segthor_original/train/Patient_01/GT.nii.gz'
# # pred_nifti_path = '../../data/segthor_original/train/Patient_02/GT.nii.gz'
# 
# gt_nifti_path = '/Users/krijndignum/Documents/Master AI/AI4MI/model_examples/GT/Patient_01.nii.gz'
# pred_nifti_path = '/Users/krijndignum/Documents/Master AI/AI4MI/model_examples/2d/corrected_GT/Focal/Patient_01.nii.gz'
# num_classes = 5  # Adjust based on your dataset (including background)
# 
# # Load and process the ground truth volume
# gt_volume = load_nifti_as_tensor(gt_nifti_path, num_classes)
# 
# # Load and process the prediction volume
# pred_volume = load_nifti_as_tensor(pred_nifti_path, num_classes)
# 
# # Now you can use gt_volume and pred_volume in your plotting functions
# volume_id = 'volume_identifier'  # Adjust as needed
# output_dir = '/Users/krijndignum/Documents/Master AI/AI4MI/model_examples/plot_output'
# os.makedirs(output_dir, exist_ok=True)
# render_3d_segmentation(pred_volume, gt_volume, volume_id, output_dir=output_dir)


# Plot 3d visualizations of 'affine', 'combined', 'corrected_GT', 'CRF', 'elastic', 'noise'

In [16]:
def automate_patient_1_structure(predictions_base_dir, gt_base_dir, output_base_dir, volume_id):
    # Loop through different types (including CRF, 3D, and 2D)
    types = ['affine', 'combined', 'corrected_GT', 'CRF', 'elastic', 'noise']
    
    # Process for 2D data
    for t in types:
        if t == 'corrected_GT':
            # Handle subfolders inside corrected_GT
            subfolders = ['CE_loss', 'Dice_CE_loss', 'Dice_loss', 'Focal']
            for subfolder in subfolders:
                pred_dir = os.path.join(predictions_base_dir, '2d', t, subfolder)
                output_dir = os.path.join(output_base_dir, '2d', t, subfolder)
                os.makedirs(output_dir, exist_ok=True)

                pred_file = os.path.join(pred_dir, 'Patient_01.nii.gz')
                gt_file = os.path.join(gt_base_dir, 'Patient_01.nii.gz')

                # Load and process the ground truth and prediction volumes
                gt_volume = load_nifti_as_tensor(gt_file, num_classes=5)
                pred_volume = load_nifti_as_tensor(pred_file, num_classes=5)

                # Render and save the 3D segmentation plots
                render_3d_segmentation(pred_volume, gt_volume, volume_id='Patient_01', output_dir=output_dir, view_type=f'{t}_{subfolder}')
        else:
            # Handle other folder types (affine, CRF, elastic, etc.) for 2D
            pred_dir = os.path.join(predictions_base_dir, '2d', t)
            output_dir = os.path.join(output_base_dir, '2d', t)
            os.makedirs(output_dir, exist_ok=True)

            pred_file = os.path.join(pred_dir, 'Patient_01.nii.gz')
            gt_file = os.path.join(gt_base_dir, 'Patient_01.nii.gz')

            # Load and process the ground truth and prediction volumes
            gt_volume = load_nifti_as_tensor(gt_file, num_classes=5)
            pred_volume = load_nifti_as_tensor(pred_file, num_classes=5)

            # Render and save the 3D segmentation plots
            render_3d_segmentation(pred_volume, gt_volume, volume_id='Patient_01', output_dir=output_dir, view_type=t)
    
    # Process for 3D data
    for t in types:
        if t == 'corrected_GT':
            subfolders = ['CE_loss', 'Dice_CE_loss', 'Dice_loss', 'Focal']
            for subfolder in subfolders:
                pred_dir = os.path.join(predictions_base_dir, '3d', t, subfolder)
                output_dir = os.path.join(output_base_dir, '3d', t, subfolder)
                os.makedirs(output_dir, exist_ok=True)
                
                pred_file = os.path.join(pred_dir, 'Patient_01.nii.gz')
                gt_file = os.path.join(gt_base_dir, 'Patient_01.nii.gz')

                gt_volume = load_nifti_as_tensor(gt_file, num_classes=5)
                pred_volume = load_nifti_as_tensor(pred_file, num_classes=5)
                
                render_3d_segmentation(pred_volume, gt_volume, volume_id='Patient_01', output_dir=output_dir, view_type=f'{t}_{subfolder}')
        else:
            pred_dir = os.path.join(predictions_base_dir, '3d', t)
            output_dir = os.path.join(output_base_dir, '3d', t)
            os.makedirs(output_dir, exist_ok=True)
            
            pred_file = os.path.join(pred_dir, 'Patient_01.nii.gz')
            gt_file = os.path.join(gt_base_dir, 'Patient_01.nii.gz')

            gt_volume = load_nifti_as_tensor(gt_file, num_classes=5)
            pred_volume = load_nifti_as_tensor(pred_file, num_classes=5)
            
            render_3d_segmentation(pred_volume, gt_volume, volume_id='Patient_01', output_dir=output_dir, view_type=t)

# Set base directories
predictions_base_dir = '/Users/krijndignum/Documents/Master AI/AI4MI/model_examples'
gt_base_dir = '/Users/krijndignum/Documents/Master AI/AI4MI/model_examples/GT'
output_base_dir = '/Users/krijndignum/Documents/Master AI/AI4MI/model_examples/plot_output'

# Automate for patient 1
automate_patient_1_structure(predictions_base_dir, gt_base_dir, output_base_dir, volume_id='Patient_01')


Loaded NIfTI file from /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/GT/Patient_01.nii.gz
Original volume shape: (512, 512, 229)
Data type: float64
Voxel sizes (after reorientation): (np.float32(0.9765625), np.float32(0.9765625), np.float32(2.0))
Axes codes after reorientation: ('R', 'A', 'S')
Volume shape after transpose: (229, 512, 512)
Scaling factors: (np.float32(2.048), np.float32(1.0), np.float32(1.0))
Volume shape after resampling: (469, 512, 512)
Unique labels in the resampled volume: [0 1 2 3 4]
Loaded NIfTI file from /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/3d/affine/Patient_01.nii.gz
Original volume shape: (512, 512, 229)
Data type: float64
Voxel sizes (after reorientation): (np.float32(0.9765625), np.float32(0.9765625), np.float32(2.0))
Axes codes after reorientation: ('R', 'A', 'S')
Volume shape after transpose: (229, 512, 512)
Scaling factors: (np.float32(2.048), np.float32(1.0), np.float32(1.0))
Volume shape after resampling: (469, 512, 5

2024-10-24 08:37:46.244 (11225.078s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 08:37:51.119 (11229.954s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 08:37:55.964 (11234.798s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 08:38:00.689 (11239.523s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 08:38:05.704 (11244.539s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 08:38:10.546 (11249.380s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 08:38:15.475 (11254.309s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 08:38:20.277 (11259.111s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type


Saved isometric view to /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/plot_output/3d/affine/Patient_01_affine_isometric_view.png
Saved front view to /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/plot_output/3d/affine/Patient_01_affine_front_view.png
Saved side view to /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/plot_output/3d/affine/Patient_01_affine_side_view.png
Saved top view to /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/plot_output/3d/affine/Patient_01_affine_top_view.png
Rendered 3D segmentation saved to /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/plot_output/3d/affine/Patient_01_affine_top_view.png
Loaded NIfTI file from /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/GT/Patient_01.nii.gz
Original volume shape: (512, 512, 229)
Data type: float64
Voxel sizes (after reorientation): (np.float32(0.9765625), np.float32(0.9765625), np.float32(2.0))
Axes codes after reorientation: ('R', 'A', 'S')
Vo

2024-10-24 08:39:18.262 (11317.095s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 08:39:23.052 (11321.886s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 08:39:27.921 (11326.755s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 08:39:32.651 (11331.484s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 08:39:37.695 (11336.529s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 08:39:42.523 (11341.357s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 08:39:47.517 (11346.351s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 08:39:52.283 (11351.117s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type


Saved isometric view to /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/plot_output/3d/combined/Patient_01_combined_isometric_view.png
Saved front view to /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/plot_output/3d/combined/Patient_01_combined_front_view.png
Saved side view to /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/plot_output/3d/combined/Patient_01_combined_side_view.png
Saved top view to /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/plot_output/3d/combined/Patient_01_combined_top_view.png
Rendered 3D segmentation saved to /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/plot_output/3d/combined/Patient_01_combined_top_view.png
Loaded NIfTI file from /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/GT/Patient_01.nii.gz
Original volume shape: (512, 512, 229)
Data type: float64
Voxel sizes (after reorientation): (np.float32(0.9765625), np.float32(0.9765625), np.float32(2.0))
Axes codes after reorientation

...

2024-10-24 08:48:35.027 (11873.857s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 08:48:39.924 (11878.754s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 08:48:44.867 (11883.697s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 08:48:49.633 (11888.463s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 08:48:54.674 (11893.504s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 08:48:59.519 (11898.349s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 08:49:04.381 (11903.211s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 08:49:09.161 (11907.991s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type


Saved isometric view to /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/plot_output/3d/elastic/Patient_01_elastic_isometric_view.png
Saved front view to /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/plot_output/3d/elastic/Patient_01_elastic_front_view.png
Saved side view to /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/plot_output/3d/elastic/Patient_01_elastic_side_view.png
Saved top view to /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/plot_output/3d/elastic/Patient_01_elastic_top_view.png
Rendered 3D segmentation saved to /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/plot_output/3d/elastic/Patient_01_elastic_top_view.png
Loaded NIfTI file from /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/GT/Patient_01.nii.gz
Original volume shape: (512, 512, 229)
Data type: float64
Voxel sizes (after reorientation): (np.float32(0.9765625), np.float32(0.9765625), np.float32(2.0))
Axes codes after reorientation: ('R', 'A

2024-10-24 08:50:06.778 (11965.607s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 08:50:11.576 (11970.405s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 08:50:16.439 (11975.268s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 08:50:21.190 (11980.020s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 08:50:26.205 (11985.035s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 08:50:31.000 (11989.830s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 08:50:35.877 (11994.707s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 08:50:40.642 (11999.472s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type


Saved isometric view to /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/plot_output/3d/noise/Patient_01_noise_isometric_view.png
Saved front view to /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/plot_output/3d/noise/Patient_01_noise_front_view.png
Saved side view to /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/plot_output/3d/noise/Patient_01_noise_side_view.png
Saved top view to /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/plot_output/3d/noise/Patient_01_noise_top_view.png
Rendered 3D segmentation saved to /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/plot_output/3d/noise/Patient_01_noise_top_view.png
Loaded NIfTI file from /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/GT/Patient_01.nii.gz
Original volume shape: (512, 512, 229)
Data type: float64
Voxel sizes (after reorientation): (np.float32(0.9765625), np.float32(0.9765625), np.float32(2.0))
Axes codes after reorientation: ('R', 'A', 'S')
Volume shape

FileNotFoundError: No such file or no access: '/Users/krijndignum/Documents/Master AI/AI4MI/model_examples/2d/3d/affine/Patient_01.nii.gz'

# code for tversky 2d and 3d

In [7]:
import os
import pyvista as pv
import torch
import numpy as np

custom_palette = {
    'roze': '#eb8fd8',
    'groen': '#b9d4b4',
    'paars': '#ba94e9',
    'blue': '#4C8BE2',
    'orange': '#E27A3F',
    'grey_light': '#1F3240',
    'grey_dark': '#16242F'
}

def create_structured_grid(volume_np):
    W, H, D = volume_np.shape
    x = np.arange(W)
    y = np.arange(H)
    z = np.arange(D)
    X, Y, Z = np.meshgrid(x, y, z, indexing='ij')
    grid = pv.StructuredGrid(X, Y, Z)
    grid.point_data["labels"] = volume_np.flatten(order="F")
    return grid

def render_3d_segmentation_tversky(pred_volume, gt_volume, volume_id, output_dir):
    pred_volume_np = torch.argmax(pred_volume, dim=0).cpu().numpy()
    gt_volume_np = torch.argmax(gt_volume, dim=0).cpu().numpy()
    pred_volume_np = np.transpose(pred_volume_np, (2, 1, 0))
    gt_volume_np = np.transpose(gt_volume_np, (2, 1, 0))

    # Check unique labels in prediction and ground truth
    pred_unique_labels = np.unique(pred_volume_np)
    gt_unique_labels = np.unique(gt_volume_np)
    print(f"Unique labels in prediction for Tversky: {pred_unique_labels}")
    print(f"Unique labels in ground truth for Tversky: {gt_unique_labels}")

    grid_pred = create_structured_grid(pred_volume_np)
    grid_gt = create_structured_grid(gt_volume_np)

    class_names = ['Background', 'Esophagus', 'Heart', 'Trachea', 'Aorta']
    class_colors = ['gray', custom_palette['roze'], custom_palette['groen'], custom_palette['paars'], custom_palette['blue']]

    p = pv.Plotter(shape=(1, 2), window_size=(1600, 800), off_screen=True)

    def add_class_surfaces(grid, subplot_index, title, view_type, unique_labels):
        p.subplot(0, subplot_index)
        for c in range(1, len(class_names)):  # Skipping background if needed
            class_label = c
            if class_label not in unique_labels:
                print(f"Skipping class {class_names[c]} for {view_type} because it has no points.")
                continue
            class_grid = grid.threshold(value=(class_label - 0.1, class_label + 0.1), scalars='labels')
            if class_grid.n_points == 0:
                print(f"No points found for class {class_names[c]} in {view_type}")
                continue  # Skip if the mesh has no points
            surface = class_grid.contour(isosurfaces=[class_label], scalars='labels')
            if surface.n_points == 0:
                print(f"Skipping empty mesh for class {class_names[c]} in {view_type}")
                continue
            p.add_mesh(surface, color=class_colors[c], opacity=0.6, label=class_names[c])
        p.add_legend(bcolor='white', loc='lower right' )
        p.add_axes()
        p.set_background('white')
        p.add_title(f'{title} - {view_type}')

    # Add ground truth and prediction surfaces, skipping missing labels
    add_class_surfaces(grid_gt, subplot_index=0, title='Ground Truth - Tversky', view_type="Ground Truth", unique_labels=gt_unique_labels)
    add_class_surfaces(grid_pred, subplot_index=1, title='Prediction - Tversky', view_type="Prediction", unique_labels=pred_unique_labels)
    
    p.link_views()

    camera_positions = {'isometric': 'iso', 'front': 'xz', 'side': 'yz', 'top': 'xy'}
    for view_name, camera_pos in camera_positions.items():
        p.camera_position = camera_pos
        p.render()
        screenshot_path = os.path.join(output_dir, f'{volume_id}_Tversky_{view_name}_view.png')
        p.screenshot(screenshot_path)
        print(f"Saved {view_name} view to {screenshot_path}")

    p.close()

# Set paths for Tversky
pred_file = '/Users/krijndignum/Documents/Master AI/AI4MI/model_examples/2d/corrected_GT/Tversky/Patient_01.nii.gz'
gt_file = '/Users/krijndignum/Documents/Master AI/AI4MI/model_examples/GT/Patient_01.nii.gz'
output_dir = '/Users/krijndignum/Documents/Master AI/AI4MI/model_examples/plot_output/2d/corrected_GT/Tversky'
os.makedirs(output_dir, exist_ok=True)

# Load and process the ground truth and prediction volumes
gt_volume = load_nifti_as_tensor(gt_file, num_classes=5)
pred_volume = load_nifti_as_tensor(pred_file, num_classes=5)

# Render and save the 3D segmentation plots for Tversky
render_3d_segmentation_tversky(pred_volume, gt_volume, volume_id='Patient_01', output_dir=output_dir)


# Set paths for 3D Tversky
pred_file = '/Users/krijndignum/Documents/Master AI/AI4MI/model_examples/3d/corrected_GT/Tversky/Patient_01.nii.gz'
gt_file = '/Users/krijndignum/Documents/Master AI/AI4MI/model_examples/GT/Patient_01.nii.gz'
output_dir = '/Users/krijndignum/Documents/Master AI/AI4MI/model_examples/plot_output/3d/corrected_GT/Tversky'
os.makedirs(output_dir, exist_ok=True)

# Load and process the ground truth and prediction volumes
gt_volume = load_nifti_as_tensor(gt_file, num_classes=5)
pred_volume = load_nifti_as_tensor(pred_file, num_classes=5)

# Render and save the 3D segmentation plots for Tversky
render_3d_segmentation_tversky(pred_volume, gt_volume, volume_id='Patient_01', output_dir=output_dir)

Loaded NIfTI file from /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/GT/Patient_01.nii.gz
Original volume shape: (512, 512, 229)
Data type: float64
Voxel sizes (after reorientation): (np.float32(0.9765625), np.float32(0.9765625), np.float32(2.0))
Axes codes after reorientation: ('R', 'A', 'S')
Volume shape after transpose: (229, 512, 512)
Scaling factors: (np.float32(2.048), np.float32(1.0), np.float32(1.0))
Volume shape after resampling: (469, 512, 512)
Unique labels in the resampled volume: [0 1 2 3 4]
Loaded NIfTI file from /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/2d/corrected_GT/Tversky/Patient_01.nii.gz
Original volume shape: (512, 512, 229)
Data type: float64
Voxel sizes (after reorientation): (np.float32(0.9765625), np.float32(0.9765625), np.float32(2.0))
Axes codes after reorientation: ('R', 'A', 'S')
Volume shape after transpose: (229, 512, 512)
Scaling factors: (np.float32(2.048), np.float32(1.0), np.float32(1.0))
Volume shape after resampling

2024-10-24 05:43:24.166 ( 763.016s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 05:43:29.026 ( 767.876s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 05:43:33.973 ( 772.822s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 05:43:38.990 ( 777.840s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 05:43:53.022 ( 791.872s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type


Skipping empty mesh for class Esophagus in Prediction


2024-10-24 05:44:18.288 ( 817.138s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 05:44:23.169 ( 822.019s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type
2024-10-24 05:44:27.909 ( 826.759s) [           B81FD]vtkContour3DLinearGrid.:1776  INFO| Invalid scalar array type


Saved isometric view to /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/plot_output/2d/corrected_GT/Tversky/Patient_01_Tversky_isometric_view.png
Saved front view to /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/plot_output/2d/corrected_GT/Tversky/Patient_01_Tversky_front_view.png
Saved side view to /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/plot_output/2d/corrected_GT/Tversky/Patient_01_Tversky_side_view.png
Saved top view to /Users/krijndignum/Documents/Master AI/AI4MI/model_examples/plot_output/2d/corrected_GT/Tversky/Patient_01_Tversky_top_view.png


In [ ]:
# THIS IS ONLY USED FOR PICKLE OBJECTS OF THE PREDICTIONS IN VOLUME

# def animate_3d_volume(volume_predictions, volume_ground_truths, evaluate_dir):
#     # Assuming volume_predictions and volume_ground_truths are defined as before
#     ids = [1, 13, 22, 28, 30]
#     for volume_id in ids:
#         # Retrieve the list of slices for the given volume_id
#         pred_slices = volume_predictions[volume_id]  # List of (slice_idx, pred_slice)
#         gt_slices = volume_ground_truths[volume_id]  # List of (slice_idx, gt_slice)
# 
#         # Sort the slices by slice_idx
#         pred_slices_sorted = sorted(pred_slices, key=lambda x: x[0])
#         gt_slices_sorted = sorted(gt_slices, key=lambda x: x[0])
# 
#         # Stack the slices into a Tensor volume
#         pred_volume = torch.stack([slice_data for idx, slice_data in pred_slices_sorted], dim=1)
#         gt_volume = torch.stack([slice_data for idx, slice_data in gt_slices_sorted], dim=1)
# 
#         # Now pred_volume and gt_volume are Tensors of shape (K, D, H, W)
#         # where K is the number of classes, D is the depth (number of slices)
# 
#         # Call the rendering function
#         # render_3d_segmentation(pred_volume, gt_volume, volume_id)
#         render_3d_segmentation(pred_volume, gt_volume, volume_id, output_dir=evaluate_dir)

In [ ]:
# ANIMATION CODE

# import torch
# import pyvista as pv
# import numpy as np
# import os
# 
# def render_3d_segmentation_with_animation(pred_volume, gt_volume, volume_id, output_dir):
#     # pred_volume and gt_volume: Tensors of shape (K, D, H, W)
# 
#     # Convert to numpy arrays and get label volumes
#     pred_volume_np = torch.argmax(pred_volume, dim=0).cpu().numpy()  # Shape: (D, H, W)
#     gt_volume_np = torch.argmax(gt_volume, dim=0).cpu().numpy()
# 
#     # Transpose the volumes to match PyVista's (X, Y, Z) format
#     pred_volume_np = np.transpose(pred_volume_np, (2, 1, 0))  # Now shape is (W, H, D)
#     gt_volume_np = np.transpose(gt_volume_np, (2, 1, 0))
# 
#     # Create the grids
#     grid_pred = pv.UniformGrid()
#     grid_pred.dimensions = pred_volume_np.shape
#     grid_pred.spacing = (1, 1, 1)
#     grid_pred.origin = (0, 0, 0)
#     grid_pred.point_data["labels"] = pred_volume_np.flatten(order="F")
# 
#     grid_gt = pv.UniformGrid()
#     grid_gt.dimensions = gt_volume_np.shape
#     grid_gt.spacing = (1, 1, 1)
#     grid_gt.origin = (0, 0, 0)
#     grid_gt.point_data["labels"] = gt_volume_np.flatten(order="F")
# 
#     # Define class names and colors (adjust as needed)
#     class_names = ['Background', 'Esophagus', 'Heart', 'Trachea', 'Aorta']
#     class_colors = ['gray', 'red', 'green', 'blue', 'yellow']
# 
#     # Create the plotter with subplots
#     p = pv.Plotter(shape=(1, 2), off_screen=True, window_size=(1600, 800))
# 
#     # Function to add class surfaces to a subplot
#     def add_class_surfaces(grid, subplot_index, title):
#         p.subplot(0, subplot_index)
#         for c in range(1, len(class_names)):  # Skip background if desired
#             class_label = c
#             # Threshold the grid to extract the class
#             class_grid = grid.threshold(value=(class_label - 0.1, class_label + 0.1), scalars='labels')
#             if class_grid.n_points == 0:
#                 continue  # Skip if no points for this class
#             # Extract the surface mesh
#             surface = class_grid.contour(isosurfaces=[class_label], scalars='labels')
#             # Add the mesh to the plotter
#             p.add_mesh(surface, color=class_colors[c], opacity=0.6, label=class_names[c])
#         p.add_legend(bcolor='white')
#         p.add_axes()
#         p.set_background('white')
#         p.add_title(title)
# 
#     # Add ground truth to the first subplot
#     add_class_surfaces(grid_gt, subplot_index=0, title='Ground Truth')
# 
#     # Add prediction to the second subplot
#     add_class_surfaces(grid_pred, subplot_index=1, title='Prediction')
# 
#     # Link the views so that camera movements affect both subplots
#     p.link_views()
# 
#     # Set the camera position
#     p.camera_position = 'iso'  # Isometric view
# 
#     # Prepare to capture frames
#     num_frames = 60
#     rotation_angle = 360 / num_frames
# 
#     # Directory to save frames
#     frames_dir = os.path.join(output_dir, f'volume_{volume_id}_frames')
#     os.makedirs(frames_dir, exist_ok=True)
# 
#     # Capture frames
#     for i in range(num_frames):
#         # Rotate the camera
#         p.camera.Azimuth(rotation_angle)
#         # Update rendering
#         p.render()
#         # Save frame
#         frame_path = os.path.join(frames_dir, f'frame_{i:03d}.png')
#         p.screenshot(frame_path)
# 
#     # Close the plotter
#     p.close()
# 
#     # Create an animation (e.g., GIF)
#     animation_path = os.path.join(output_dir, f'volume_{volume_id}_animation.gif')
#     import imageio
#     with imageio.get_writer(animation_path, mode='I', duration=0.05) as writer:
#         for i in range(num_frames):
#             frame_path = os.path.join(frames_dir, f'frame_{i:03d}.png')
#             image = imageio.imread(frame_path)
#             writer.append_data(image)
# 
#     print(f"Animation saved to {animation_path}")
# 
#     # Optionally, remove the frames directory to save space
#     import shutil
#     shutil.rmtree(frames_dir)
